Primeiro, converti manualmente as tabelas que estavam fomatadas com Tabulate do python, para uma cadeia de strings para serem gravadas em arquivos csv cada. Devido ao tamanho pequeno e simplicidade da tabela, resolvi fazer o processo manualmente para maior eficiência do código.

In [186]:
tabela_clientes = '''CD_CLIENTE,NM_CLIENTE
1,João
2,Maria
3,José
4,Adilson
5,Cleber
'''
tabela_transacoes =  '''CD_CLIENTE,DT_TRANSACAO,CD_TRANSACAO,VR_TRANSACAO
1,2021-08-28,000,020.00
1,2021-09-09,110,078.90
1,2021-09-17,220,058.00
1,2021-11-15,110,178.90
1,2021-12-24,220,110.37
5,2021-10-28,110,220.00
5,2021-11-07,110,380.00
5,2021-12-05,220,398.86
5,2021-12-14,220,033.90
5,2021-12-21,220,016.90
3,2021-10-05,110,720.90
3,2021-11-05,110,720.90
3,2021-12-05,110,720.90
4,2021-10-09,000,050.00
'''

with open("cliente.csv", "w", encoding="utf-8") as f:
    f.write(tabela_clientes)

with open("transacoes.csv", "w", encoding="utf-8") as f:
    f.write(tabela_transacoes)

Agora, podemos começar a tratar do código e importar os arquivos para serem utilizados no pandas.

In [83]:
import pandas as pd

clientes_df = pd.read_csv('cliente.csv')
transacoes_df = pd.read_csv('transacoes.csv')


Transformando a coluna de datas no formato datetime e transformando valores de CashOut em negativo para facilitar operações.

In [198]:
transacoes_df['DT_TRANSACAO'] = pd.to_datetime(transacoes_df['DT_TRANSACAO'])

transacoes_df['VR_TRANSACAO'] = transacoes_df['VR_TRANSACAO'].where(transacoes_df['CD_TRANSACAO'] != 220, -(transacoes_df['VR_TRANSACAO']))
print(transacoes_df['VR_TRANSACAO'])


0      20.00
1      78.90
2     -58.00
3     178.90
4    -110.37
5     220.00
6     380.00
7    -398.86
8     -33.90
9     -16.90
10    720.90
11    720.90
12    720.90
13     50.00
Name: VR_TRANSACAO, dtype: float64


--Qual cliente teve o maior saldo médio no mês 11?

In [200]:
saldo_medio_cliente = transacoes_df.groupby('CD_CLIENTE')['VR_TRANSACAO'].mean()

novembro_df = transacoes_df[transacoes_df['DT_TRANSACAO'].dt.month == 11]
saldo_medio_novembro = saldo_medio_cliente.loc[novembro_df['CD_CLIENTE']]
id_cliente_maior_saldo = saldo_medio_novembro.idxmax()
maior_saldo = saldo_medio_novembro.max()

nome_cliente = clientes_df[clientes_df['CD_CLIENTE'] == id_cliente_maior_saldo]['NM_CLIENTE'].iloc[0]

print(f'O cliente com o maior saldo médio em novembro é {nome_cliente} com $ {maior_saldo}')

O cliente com o maior saldo médio em novembro é José com $ 720.9


--Qual é o saldo de cada cliente?

In [202]:
saldo_cliente = transacoes_df.groupby('CD_CLIENTE')['VR_TRANSACAO'].sum()
for cliente, saldo in saldo_cliente.items():
    nome_cliente = clientes_df[clientes_df['CD_CLIENTE'] == cliente]['NM_CLIENTE'].iloc[0]
    print(f'O cliente {nome_cliente} possui um saldo de ${saldo:.2f}')



O cliente João possui um saldo de $109.43
O cliente José possui um saldo de $2162.70
O cliente Adilson possui um saldo de $50.00
O cliente Cleber possui um saldo de $150.34


--Qual é o saldo médio de clientes que receberam CashBack?

In [203]:
cd_cliente_cashback = transacoes_df.query('CD_TRANSACAO == 000')['CD_CLIENTE']
saldo_medio_cashback = (saldo_medio_cliente.loc[cd_cliente_cashback]).mean()
print(f'O saldo médio dos clientes que receberam cashback é de ${saldo_medio_cashback:.2f}')

O saldo médio dos clientes que receberam cashback é de $35.94


--Qual o ticket médio das quatro últimas movimentações dos usuários?

In [204]:
transacoes_por_data_df = transacoes_df.sort_values(by='DT_TRANSACAO', ascending=False)
ticket_medio= transacoes_por_data_df.groupby('CD_CLIENTE')['VR_TRANSACAO'].head(4).mean()
print(f'O ticket médio para as quatro últimas movimentações é de ${ticket_medio:.2f}')


O ticket médio para as quatro últimas movimentações é de $186.04


--Qual é a proporção entre Cash In/Out mensal?

In [205]:

transacoes_cashin = transacoes_df.query('CD_TRANSACAO == 110')[['CD_TRANSACAO']].groupby(transacoes_df['DT_TRANSACAO'].dt.month)['CD_TRANSACAO'].count()
transacoes_cashout = transacoes_df.query('CD_TRANSACAO == 220')[['CD_TRANSACAO']].groupby(transacoes_df['DT_TRANSACAO'].dt.month)['CD_TRANSACAO'].count()

total_transacoes = transacoes_cashin + transacoes_cashout

proporcao = (transacoes_cashin / total_transacoes * 100).reset_index().fillna(100)
proporcao.columns = ['MES', 'PORCENTAGEM_CASHIN']
proporcao['PORCENTAGEM_CASHOUT'] = 100 - proporcao['PORCENTAGEM_CASHIN']
print(proporcao)


   MES  PORCENTAGEM_CASHIN  PORCENTAGEM_CASHOUT
0    9                50.0                 50.0
1   10               100.0                  0.0
2   11               100.0                  0.0
3   12                20.0                 80.0


--Qual a última transação de cada tipo para cada usuário?

In [206]:
ultimas_transacoes = transacoes_df.sort_values('DT_TRANSACAO', ascending=False).groupby(['CD_CLIENTE', 'CD_TRANSACAO']).first()
ultimas_transacoes = ultimas_transacoes.reset_index()

tipo_transacao = {
        110: 'CashIn',
        220: 'CashOut',
        000: 'CashBack'
    }

ultimas_transacoes['CD_TRANSACAO'] = ultimas_transacoes['CD_TRANSACAO'].replace(tipo_transacao)


for cliente, transacoes in ultimas_transacoes.groupby('CD_CLIENTE'):
    nome_cliente = clientes_df[clientes_df['CD_CLIENTE'] == cliente]['NM_CLIENTE'].iloc[0]
    print(f'\n Cliente {nome_cliente}:')
    print(transacoes[['CD_TRANSACAO', 'DT_TRANSACAO', 'VR_TRANSACAO']])




 Cliente João:
  CD_TRANSACAO DT_TRANSACAO  VR_TRANSACAO
0     CashBack   2021-08-28         20.00
1       CashIn   2021-11-15        178.90
2      CashOut   2021-12-24       -110.37

 Cliente José:
  CD_TRANSACAO DT_TRANSACAO  VR_TRANSACAO
3       CashIn   2021-12-05         720.9

 Cliente Adilson:
  CD_TRANSACAO DT_TRANSACAO  VR_TRANSACAO
4     CashBack   2021-10-09          50.0

 Cliente Cleber:
  CD_TRANSACAO DT_TRANSACAO  VR_TRANSACAO
5       CashIn   2021-11-07         380.0
6      CashOut   2021-12-21         -16.9


--Qual a última transação de cada tipo para cada usuário por mês?

In [207]:
ultimas_transacoes_por_mes = transacoes_df.sort_values('DT_TRANSACAO', ascending=False)
ultimas_transacoes_por_mes = transacoes_df.groupby(['CD_CLIENTE', transacoes_df['DT_TRANSACAO'].dt.month]).first()
tipo_transacao = {
        110: 'CashIn',
        220: 'CashOut',
        000: 'CashBack'
    }

ultimas_transacoes_por_mes['CD_TRANSACAO'] = ultimas_transacoes_por_mes['CD_TRANSACAO'].replace(tipo_transacao)

for cliente, transacoes in ultimas_transacoes_por_mes.groupby('CD_CLIENTE'):
    nome_cliente = clientes_df[clientes_df['CD_CLIENTE'] == cliente]['NM_CLIENTE'].iloc[0]
    print(f'\n Cliente {nome_cliente}:')
    print(transacoes[['CD_TRANSACAO', 'DT_TRANSACAO', 'VR_TRANSACAO']])




 Cliente João:
                        CD_TRANSACAO DT_TRANSACAO  VR_TRANSACAO
CD_CLIENTE DT_TRANSACAO                                        
1          8                CashBack   2021-08-28         20.00
           9                  CashIn   2021-09-09         78.90
           11                 CashIn   2021-11-15        178.90
           12                CashOut   2021-12-24       -110.37

 Cliente José:
                        CD_TRANSACAO DT_TRANSACAO  VR_TRANSACAO
CD_CLIENTE DT_TRANSACAO                                        
3          10                 CashIn   2021-10-05         720.9
           11                 CashIn   2021-11-05         720.9
           12                 CashIn   2021-12-05         720.9

 Cliente Adilson:
                        CD_TRANSACAO DT_TRANSACAO  VR_TRANSACAO
CD_CLIENTE DT_TRANSACAO                                        
4          10               CashBack   2021-10-09          50.0

 Cliente Cleber:
                        CD_TRANSACA

--Qual quatidade de usuários que movimentaram a conta?

In [208]:
num_clientes = transacoes_df['CD_CLIENTE'].nunique()
print(f"Quantidade de usuários que movimentaram a conta: {num_clientes}")


Quantidade de usuários que movimentaram a conta: 4


--Qual o balanço do final de 2021?

In [209]:
balaco_2021 = transacoes_df[transacoes_df['DT_TRANSACAO'].dt.year == 2021]
balaco_2021 = balaco_2021['VR_TRANSACAO'].sum()
print(f'O balanço final de 2021 foi de ${balaco_2021:.2f}')


O balanço final de 2021 foi de $2472.47


--Quantos usuários que receberam CashBack continuaram interagindo com este banco?

In [210]:

cleintes_cashback = transacoes_df[transacoes_df['CD_TRANSACAO'] == 000]['CD_CLIENTE'].unique()
interacao = transacoes_df[transacoes_df['CD_CLIENTE'].isin(cleintes_cashback) & (transacoes_df['CD_TRANSACAO'] != 0)]['CD_CLIENTE'].nunique()

print(f'{interacao} usuários que receberam CashBack continuaram interagindo com o banco.')


1 usuários que receberam CashBack continuaram interagindo com o banco.


--Qual a primeira e a última movimentação dos usuários com saldo maior que R$100?

In [211]:
maior_que_100 = transacoes_df.query('VR_TRANSACAO > 100')
primeira_transacao = maior_que_100.sort_values('DT_TRANSACAO').groupby('CD_CLIENTE').first()
ultima_transacao = maior_que_100.sort_values('DT_TRANSACAO', ascending=False).groupby('CD_CLIENTE').first()

for cliente in primeira_transacao.index:
    nome_cliente = clientes_df[clientes_df['CD_CLIENTE'] == cliente]['NM_CLIENTE'].iloc[0]
    ultima_transacao_cliente = ultima_transacao.loc[cliente].replace(tipo_transacao).reset_index()
    primeira_transacao_cliente = primeira_transacao.loc[cliente].replace(tipo_transacao).reset_index()
    print(f'\n Cliente {nome_cliente}:')
    print(f'Primeira transação:', primeira_transacao_cliente)
    print(f'Ultima transação:', ultima_transacao_cliente)




 Cliente João:
Primeira transação:           index                    1
0  DT_TRANSACAO  2021-11-15 00:00:00
1  CD_TRANSACAO               CashIn
2  VR_TRANSACAO                178.9
Ultima transação:           index                    1
0  DT_TRANSACAO  2021-11-15 00:00:00
1  CD_TRANSACAO               CashIn
2  VR_TRANSACAO                178.9

 Cliente José:
Primeira transação:           index                    3
0  DT_TRANSACAO  2021-10-05 00:00:00
1  CD_TRANSACAO               CashIn
2  VR_TRANSACAO                720.9
Ultima transação:           index                    3
0  DT_TRANSACAO  2021-12-05 00:00:00
1  CD_TRANSACAO               CashIn
2  VR_TRANSACAO                720.9

 Cliente Cleber:
Primeira transação:           index                    5
0  DT_TRANSACAO  2021-10-28 00:00:00
1  CD_TRANSACAO               CashIn
2  VR_TRANSACAO              CashOut
Ultima transação:           index                    5
0  DT_TRANSACAO  2021-11-07 00:00:00
1  CD_TRANSACAO       

--Qual o balanço das últimas quatro movimentações de cada usuário?

In [212]:
transacoes_por_data_df = transacoes_df.sort_values(by='DT_TRANSACAO', ascending=False)

for cliente, transacoes in transacoes_por_data_df.groupby('CD_CLIENTE'):
    nome_cliente = clientes_df[clientes_df['CD_CLIENTE'] == cliente]['NM_CLIENTE'].iloc[0]
    ultimas_4 = transacoes[['CD_TRANSACAO', 'DT_TRANSACAO', 'VR_TRANSACAO']].head(4)
    balanco_cliente = ultimas_4['VR_TRANSACAO'].sum()
    print(f'\n O balanço das ultimas quatro movimentações do Cliente {nome_cliente} é ${balanco_cliente:.2f}')






 O balanço das ultimas quatro movimentações do Cliente João é $89.43

 O balanço das ultimas quatro movimentações do Cliente José é $2162.70

 O balanço das ultimas quatro movimentações do Cliente Adilson é $50.00

 O balanço das ultimas quatro movimentações do Cliente Cleber é $-69.66


--Qual o ticket médio das últimas quatro movimentações de cada usuário?


In [213]:
transacoes_por_data_df = transacoes_df.sort_values(by='DT_TRANSACAO', ascending=False)

for cliente, transacoes in transacoes_por_data_df.groupby('CD_CLIENTE'):
    nome_cliente = clientes_df[clientes_df['CD_CLIENTE'] == cliente]['NM_CLIENTE'].iloc[0]
    ultimas_4 = transacoes[['CD_TRANSACAO', 'DT_TRANSACAO', 'VR_TRANSACAO']].head(4)
    ticket_cliente = ultimas_4['VR_TRANSACAO'].mean()
    print(f'\n O ticket médio das ultimas quatro movimentações do Cliente {nome_cliente} é ${ticket_cliente:.2f}')






 O ticket médio das ultimas quatro movimentações do Cliente João é $22.36

 O ticket médio das ultimas quatro movimentações do Cliente José é $720.90

 O ticket médio das ultimas quatro movimentações do Cliente Adilson é $50.00

 O ticket médio das ultimas quatro movimentações do Cliente Cleber é $-17.42
